In [1]:
using MPSGE_MP, JuMP

In [2]:
M = MPSGEModel()

@sectors(M,begin
    X
    Y
    W
end)

@commodities(M,begin
    PX
    PY
    PL
    PK
    PW
end)

@consumer(M, CONS)

add_production!(M,
    X,
    ScalarNest(:t; elasticity = 0, children = 
        [ScalarOutput(PX, 120.)]
    ),
    ScalarNest(:s; elasticity = .5, children = [
        ScalarInput(PY,20),
        ScalarNest(:va; elasticity = 1, children = [
            ScalarInput(PL, 40; taxes = [Tax(CONS, 0.5)]),
            ScalarInput(PK, 60; taxes = [Tax(CONS, 0.5)])
            ])
        ]
        ) 
)

add_production!(M,
    Y,
    ScalarNest(:t; elasticity = 0, children = [
        ScalarOutput(PY, 120),
    ]),
    ScalarNest(:s; elasticity = .75, children = [
        ScalarInput(PX,20),
        ScalarNest(:va; elasticity = 1, children = [
            ScalarInput(PL, 60),
            ScalarInput(PK, 40)
        ])
    ])
)


add_production!(M,
    W,
    ScalarNest(:t; elasticity = 0, children = [
        ScalarOutput(PW, 200)
    ]),
    ScalarNest(:s, elasticity = 1, children = [
        ScalarInput(PX, 100),
        ScalarInput(PY, 100),
    ])
)

add_demand!(M,
    CONS,
    [ScalarDem(PW, 200)],
    [
        ScalarEndowment(PL,100),
        ScalarEndowment(PK,100)
    ]
)

jm = build!(M);


In [7]:
fix(PW,1)

solve!(M; :cumulative_iteration_limit => 10_000)
df = generate_report(M);

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_FADC.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             6.9173e+01             0.0e+00 (f[    6])
    1     1     0     8 2.9618e+00  1.0e+00    0.0e+00 (f[    5])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 2.9618e+00           I 0.0e+00 1.9e+00 (f[    5])
    1     1     3     3 2.4397e-02  1.0e+00 SO 0.0e+00 1.7e-02 (f[    9])
    2     1     4     4 2.5675e-06  1.0e+00 SO 0.0e+00 2.1e-06 (f[    9])
    3     1     5     5 1.0563e-13  1.0e+00 SO 0.0e+00 8.5e-14 (f[    4])

Major Iterations. . . . 3
Minor Iterations. . . . 3
Restarts. . . . . . . . 0
Crash Iteratio

Postsolved residual: 1.0563e-13


In [8]:
df

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,0.861867,1.42109e-14
2,W,0.983827,4.26326e-14
3,Y,1.10915,0.0
4,PY,0.868686,-2.84217e-14
5,PW,1.0,0.0
6,PL,0.841191,-7.10543e-14
7,PX,1.15116,-7.10543e-15
8,PK,0.787062,4.26326e-14
9,CONS,196.765,0.0
